<a href="https://colab.research.google.com/github/MarinaZRocha/Mod_Ling_Robotica/blob/main/E02_Mod_ling_179741.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Universidade Federal do Rio Grande (FURG)
### Programa de Pós-Graduação em Computação (PPGComp)
#### Disciplina: ModLing para Robótica 2025
##### Prof. Rodrigo da Silva Guerra
##### E02: Sistema de Recomendação usando Word2Vec

### Descrição do Exercício

Neste exercício, o objetivo é construir um sistema de recomendação de produtos com base no histórico de compras de um e-commerce. A premissa é que a sequência de produtos adquiridos por um cliente em uma única transação possui uma relação semântica. Por exemplo, um cliente que compra itens para uma festa de aniversário provavelmente selecionará produtos relacionados a esse tema. Utilizaremos o algoritmo Word2Vec para capturar essas relações, tratando cada produto como uma "palavra" e cada compra (carrinho de compras) como uma "frase".

Você baixará a base de dados e deverá realizar todo o processo de tratamento, treinamento do modelo e implementação das funcionalidades de recomendação.

### Instruções

O exercício é demonstrado [nessa videoaula](https://youtu.be/PKOGlgVhS_A), mas você é encorajado a tentar implementar por conta própria.

1.  **Carregamento e Pré-processamento dos Dados:**
    * Faça o download do arquivo [`retail.xlsx`](https://drive.google.com/uc?id=1NK-2z0l-qTplDJJ2SHpTVBGRP3zWAK-n).
    * Carregue os dados em um DataFrame do Pandas.
    * Realize a limpeza dos dados, removendo registros com valores nulos.
    * Converta a coluna de códigos de produtos (`StockCode`) para o formato de string.

2.  **Criação do Dicionário de Produtos:**
    * Crie um dicionário Python que mapeie os códigos dos produtos (`StockCode`) às suas respectivas descrições (`Description`). Este dicionário será utilizado para a interpretação dos resultados, permitindo a visualização das descrições dos produtos recomendados.

3.  **Preparação dos Dados para Treinamento:**
    * O histórico de compras de cada cliente deve ser tratado como uma "frase", onde cada produto é uma "palavra". Agrupe os produtos por cada `InvoiceNo`, criando uma lista de listas, onde cada lista interna representa uma compra.

4.  **Treinamento do Modelo Word2Vec:**
    * Utilize a biblioteca `gensim` para treinar um modelo Word2Vec com as "frases" (compras) criadas no passo anterior.

5.  **Desenvolvimento das Funções de Recomendação:**
    * **Função 1: Produtos Similares**
        * Implemente uma função que, ao receber um código de produto, retorne uma lista dos produtos mais similares, com base no espaço latente aprendido pelo Word2Vec. A função deve exibir a descrição do produto de entrada e as descrições dos produtos mais similares, juntamente com seus respectivos scores de similaridade.
    * **Função 2: Recomendação por Carrinho de Compras**
        * Implemente uma função que receba uma lista de códigos de produtos (simulando um carrinho de compras).
        * Calcule o vetor médio de embedding para os produtos presentes no carrinho.
        * Utilize o vetor médio para encontrar os produtos mais próximos no espaço de embedding, que serão as recomendações.
        * A função deve retornar uma lista com as descrições dos produtos recomendados, excluindo aqueles que já estão no carrinho, e seus respectivos scores de similaridade.

### Código Inicial

In [2]:
# Código para baixar os dados
!gdown https://drive.google.com/uc?id=1NK-2z0l-qTplDJJ2SHpTVBGRP3zWAK-n

Downloading...
From: https://drive.google.com/uc?id=1NK-2z0l-qTplDJJ2SHpTVBGRP3zWAK-n
To: /content/retail.xlsx
100% 23.7M/23.7M [00:00<00:00, 60.0MB/s]


In [3]:
import pandas as pd
import numpy as np


In [4]:
df = pd.read_excel('retail.xlsx')


In [21]:
df = df.dropna()
df['StockCode'] = df['StockCode'].astype(str)
df.drop_duplicates(inplace=True, subset='StockCode', keep='last')

In [22]:
product_dict = dict(zip(df['StockCode'], df['Description']))

In [23]:
product_dict['10123C']

'HEARTS WRAPPING TAPE '

In [8]:
customers = df['CustomerID'].unique().tolist()
len(customers)

952

In [9]:
import random

np.random.shuffle(customers)

In [10]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
107,536381,84854,GIRLY PINK TOOL SET,5,2010-12-01 09:41:00,4.95,15311.0,United Kingdom
902,536488,21895,POTTING SHED SOW 'N' GROW SET,1,2010-12-01 12:31:00,4.25,17897.0,United Kingdom
1985,C536548,20957,PORCELAIN HANGING BELL SMALL,-1,2010-12-01 14:33:00,1.45,12472.0,Germany
2313,536576,82615,PINK MARSHMALLOW SCARF KNITTING KIT,36,2010-12-01 16:11:00,2.55,13777.0,United Kingdom
2457,536591,21488,RED WHITE SCARF HOT WATER BOTTLE,1,2010-12-01 16:58:00,3.95,14606.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [76]:
def client_history(customer_id, df):

  purchases = []
  df = df.copy()
  df["StockCode"] = df["StockCode"].astype(str)
  purchases = df.groupby(["CustomerID", "InvoiceNo"])['StockCode'].apply(list).tolist()
  # for client in customer_id:
  #   purchase = df[df['CustomerID'] == client]['StockCode'].apply(list).tolist()
  # #   #purchase = infos[infos['CustomerID'] == client]['StockCode'].tolist()
  # # #

  # #   purchase = df[df(['CustomerID'])]['StockCode'].tolist()
  #   purchases.append(purchase)

  return purchases


In [12]:
train_size = int((0.9*len(customers)))

cl_train = customers[:train_size]
cl_test = customers[train_size:len(customers)]

df_train = df[df['CustomerID'].isin(cl_train)]
df_test = df[df['CustomerID'].isin(cl_test)]

In [77]:
purchase_train = client_history(cl_train, df_train)
purchase_test = client_history(cl_test, df_test)

In [78]:
len(purchase_train)

1316

In [79]:
purchase_train[300]

['84750A']

In [2]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 14.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatib

In [1]:
import gensim
from gensim.models import Word2Vec



# Word2Vec.build_vocab(purchase_train)



In [16]:
model = Word2Vec(min_count=20,
                     window=2,
                     vector_size=300,
                     sample=6e-5,
                     alpha=0.03,
                     min_alpha=0.0007,
                     negative=20,
                     workers=10)

In [80]:
sentences = purchase_train

In [99]:
print(sentences[:20])

[['84625A', '21265', '23271'], ['21555'], ['22826'], ['16169M', '22707', '16156L', '21199', '21380', '21207', '21126', '21197'], ['21651'], ['21250'], ['21195'], ['90001B'], ['22944', '23175'], ['79190B', '79190A', '22151', '20718', '71459', '22356', '20724', '23204', '22252'], ['84598'], ['84569B'], ['35598B'], ['23198', '23184', '23166', '23165', '23004', '23002', '23000', '22998', '22994', '22720', '22693', '22670', '22667', '22666', '22628', '22467', '22114', '21908', '21874', '21872', '21533', '21248', '21216', '21181', '21175', '21162', '21159', '21158', '21154', '16014', '16008', '85152', '84596F', '84596B', '84510A', '82600', '82581', '72760B', '72232', '47559B', '47504H', '23493', '23430', '23429', '23428', '23358', '23243', '23240'], ['23571'], ['21251'], ['22929'], ['22321'], ['20957'], ['35910A']]


In [100]:
print(sentences[0])

['84625A', '21265', '23271']


In [101]:
model = Word2Vec(
    vector_size=300,
    window=2,
    min_count=1,
    negative=20,
    workers=10,
    sg=1
)

model.build_vocab(sentences)
print("Corpus count:", model.corpus_count)
print("Vocab size:", len(model.wv))

Corpus count: 1316
Vocab size: 3275


In [102]:
# train
model.train(sentences, total_examples=model.corpus_count, epochs=10)

(32750, 32750)

In [ ]:
#from gensim.models import Word2Vec

In [103]:
def similar_products(product_id):

  list_similar = model.wv.most_similar(product_id)
  print(f'Similar products to {product_dict[product_id]}:')
  for product in list_similar:
    print(f'{product_dict[product]}')


In [104]:
#test = model.wv["84874B"]
print(model.wv.most_similar('22613', topn=10))

[('23366', 0.25954365730285645), ('23567', 0.22059188783168793), ('22638', 0.21097876131534576), ('22112', 0.2107163369655609), ('84795B', 0.20648573338985443), ('23153', 0.20425768196582794), ('85179A', 0.19452616572380066), ('23058', 0.18862280249595642), ('22865', 0.18430528044700623), ('84924A', 0.18212370574474335)]


In [107]:
similar_products(22613)

KeyError: "Key '22613' not present in vocabulary"